In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Parameters

In [3]:
# Fixed parameters

DEVICE = 'ibm_torino'
_RESETS = False

backend = provider.get_backend(DEVICE)

In [4]:
# Variable parameters
import numpy as np

logicals = ['0', '1']
x_basis = [True, False]

rounds = [10, 20, 30, 40, 50, 75, 100]

print("rounds:", rounds)

rounds: [10, 20, 30, 40, 50, 75, 100]


# Find the best layout

In [5]:
from soft_info import BackendEvaluator

evaluator = BackendEvaluator(backend)
longest_path, _, _, path_info = evaluator.find_longest_good_RepCode_string(longest_length=111)

Trying RepCode string of length 111 => distance 56... Current maxs: CX: 1.0, Ancilla: 1.0


In [6]:
path_info

{'mean_gate_error': 0.006452510204489545,
 'min_gate_error': 0.0012824861837752444,
 'max_gate_error': 0.04095734989287872,
 'mean_readout_error': 0.027639639639639644,
 'min_readout_error': 0.006199999999999983,
 'max_readout_error': 0.1885,
 'mean_ancilla_error': 0.034789090909090915,
 'min_ancilla_error': 0.009400000000000075,
 'max_ancilla_error': 0.11499999999999999,
 'CX_fidelity': 0.48979442544701657}

In [7]:
link_qubits = longest_path[1::2]
code_qubits = longest_path[0::2]
layout = link_qubits + code_qubits
distance = len(code_qubits)
print("longest path:", longest_path)
print("layout:", layout)
print("distance:", distance)

longest path: [106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 91, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 74, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 53, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 36, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 15, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18, 31, 32, 33, 37, 52, 51, 50, 56, 69, 70, 71, 75, 90, 89, 88, 94, 107, 108, 109, 113, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 129]
layout: [105, 103, 101, 99, 97, 95, 76, 78, 80, 82, 84, 86, 67, 65, 63, 61, 59, 57, 38, 40, 42, 44, 46, 48, 29, 27, 25, 23, 21, 19, 0, 2, 4, 6, 8, 10, 12, 31, 33, 52, 50, 69, 71, 90, 88, 107, 109, 128, 126, 124, 122, 120, 118, 116, 114, 106, 104, 102, 100, 98, 96, 91, 77, 79, 81, 83, 85, 74, 66, 64, 62, 60, 58, 53, 39, 41, 43, 45, 47, 36, 28, 26, 24, 22, 20, 15, 1, 3, 5, 7, 9, 11, 18, 32, 37, 51, 56, 70, 75, 89, 94, 108, 113, 127, 125, 123, 121, 119, 117, 115, 129]
distance: 56


# No rounds loop

In [8]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import run_IQ_calibration
from qiskit import transpile
from Scratch import metadata_helper
from tqdm import tqdm


############################

# Round index
round_index = 6

# Z = 0
xbasis = False
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)


Submitting for rounds =  100
18:44:22 Warning: updating metadata. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 46


100%|██████████| 20/20 [04:17<00:00, 12.86s/it]


In [9]:
############################

# Z = 1
xbasis = False
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  100


100%|██████████| 20/20 [04:09<00:00, 12.45s/it]


In [10]:
############################

# X = 0
xbasis = True
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  100


100%|██████████| 20/20 [03:58<00:00, 11.90s/it]


In [11]:
############################

# X = 1
xbasis = True
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  100


100%|██████████| 20/20 [04:11<00:00, 12.58s/it]


# Round loop!

In [30]:

# Z = 0
xbasis = False
logical = '0'

# Run calibration

for T in rounds:
    code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
    qc = code.circuit[logical]
    transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 

    n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
    metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                               distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                               path_info=path_info)

    # backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
    #             job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])

# Run calibration
    

{'descr': 'subset RepCodes', 'code': 'RepetitionCodeCircuit', 'distance': 52, 'rounds': 10, 'logical': '0', 'xbasis': False, 'path_info': {'mean_gate_error': 0.007963147793478936, 'min_gate_error': 0.0034766228550469347, 'max_gate_error': 0.0232547718972346, 'mean_readout_error': 0.022770873786407766, 'min_readout_error': 0.0019000000000000128, 'max_readout_error': 0.3568, 'mean_ancilla_error': 0.02186078431372549, 'min_ancilla_error': 0.0019000000000000128, 'max_ancilla_error': 0.24750000000000005}}
{'descr': 'subset RepCodes', 'code': 'RepetitionCodeCircuit', 'distance': 52, 'rounds': 20, 'logical': '0', 'xbasis': False, 'path_info': {'mean_gate_error': 0.007963147793478936, 'min_gate_error': 0.0034766228550469347, 'max_gate_error': 0.0232547718972346, 'mean_readout_error': 0.022770873786407766, 'min_readout_error': 0.0019000000000000128, 'max_readout_error': 0.3568, 'mean_ancilla_error': 0.02186078431372549, 'min_ancilla_error': 0.0019000000000000128, 'max_ancilla_error': 0.24750000